<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/ERC20_mint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colaboratory用ERC20トークン発行プログラム
# 必要なパッケージをインストール
!pip install web3 py-solc-x

import json
import os
import solcx
from web3 import Web3
from google.colab import files
from IPython.display import display, HTML

# solcxをインストールとセットアップ
solcx.install_solc('0.8.20')
solcx.set_solc_version('0.8.20')

# ERC20トークンコントラクトのソースコード
erc20_source = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.20;

/**
 * @dev ERC20標準インターフェース
 */
interface IERC20 {
    function totalSupply() external view returns (uint256);
    function balanceOf(address account) external view returns (uint256);
    function transfer(address recipient, uint256 amount) external returns (bool);
    function allowance(address owner, address spender) external view returns (uint256);
    function approve(address spender, uint256 amount) external returns (bool);
    function transferFrom(address sender, address recipient, uint256 amount) external returns (bool);

    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);
}

/**
 * @dev ERC20トークン実装
 */
contract MyToken is IERC20 {
    string public name;
    string public symbol;
    uint8 public decimals;
    uint256 private _totalSupply;
    address public owner;

    mapping(address => uint256) private _balances;
    mapping(address => mapping(address => uint256)) private _allowances;

    constructor(
        string memory tokenName,
        string memory tokenSymbol,
        uint8 tokenDecimals,
        uint256 initialSupply
    ) {
        name = tokenName;
        symbol = tokenSymbol;
        decimals = tokenDecimals;
        owner = msg.sender;

        _totalSupply = initialSupply * (10 ** uint256(decimals));
        _balances[msg.sender] = _totalSupply;

        emit Transfer(address(0), msg.sender, _totalSupply);
    }

    function totalSupply() public view override returns (uint256) {
        return _totalSupply;
    }

    function balanceOf(address account) public view override returns (uint256) {
        return _balances[account];
    }

    function transfer(address recipient, uint256 amount) public override returns (bool) {
        _transfer(msg.sender, recipient, amount);
        return true;
    }

    function allowance(address owner_, address spender) public view override returns (uint256) {
        return _allowances[owner_][spender];
    }

    function approve(address spender, uint256 amount) public override returns (bool) {
        _approve(msg.sender, spender, amount);
        return true;
    }

    function transferFrom(address sender, address recipient, uint256 amount) public override returns (bool) {
        _transfer(sender, recipient, amount);

        uint256 currentAllowance = _allowances[sender][msg.sender];
        require(currentAllowance >= amount, "ERC20: transfer amount exceeds allowance");
        unchecked {
            _approve(sender, msg.sender, currentAllowance - amount);
        }

        return true;
    }

    function _transfer(address sender, address recipient, uint256 amount) internal {
        require(sender != address(0), "ERC20: transfer from the zero address");
        require(recipient != address(0), "ERC20: transfer to the zero address");

        uint256 senderBalance = _balances[sender];
        require(senderBalance >= amount, "ERC20: transfer amount exceeds balance");

        unchecked {
            _balances[sender] = senderBalance - amount;
        }
        _balances[recipient] += amount;

        emit Transfer(sender, recipient, amount);
    }

    function _approve(address owner_, address spender, uint256 amount) internal {
        require(owner_ != address(0), "ERC20: approve from the zero address");
        require(spender != address(0), "ERC20: approve to the zero address");

        _allowances[owner_][spender] = amount;
        emit Approval(owner_, spender, amount);
    }

    // オーナー専用の追加ミント関数
    function mint(address to, uint256 amount) public {
        require(msg.sender == owner, "Only owner can mint");
        require(to != address(0), "Mint to the zero address");

        uint256 mintAmount = amount * (10 ** uint256(decimals));
        _totalSupply += mintAmount;
        _balances[to] += mintAmount;

        emit Transfer(address(0), to, mintAmount);
    }

    // オーナー専用のバーン関数
    function burn(uint256 amount) public {
        require(msg.sender == owner, "Only owner can burn");

        uint256 burnAmount = amount * (10 ** uint256(decimals));
        uint256 accountBalance = _balances[msg.sender];
        require(accountBalance >= burnAmount, "ERC20: burn amount exceeds balance");

        unchecked {
            _balances[msg.sender] = accountBalance - burnAmount;
        }
        _totalSupply -= burnAmount;

        emit Transfer(msg.sender, address(0), burnAmount);
    }
}
'''

# ソースコードをコンパイル
def compile_contract():
    compiled_sol = solcx.compile_source(
        erc20_source,
        output_values=['abi', 'bin'],
        optimize=True,
        optimize_runs=200
    )
    contract_id, contract_interface = compiled_sol.popitem()
    return contract_interface

# メイン関数：ERC20トークンのデプロイと操作
def deploy_token():
    # Ethereumネットワーク接続設定
    print("Ethereumネットワークに接続します。使用するネットワークを選択してください：")
    print("1. ローカルGanache (http://127.0.0.1:8545)")
    print("2. Sepolia Testnet")
    print("3. Goerli Testnet")
    print("4. Ethereum Mainnet (注意: 実際のETHを使用します)")
    print("5. カスタムRPC URL")

    choice = int(input("選択肢の番号を入力してください (1-5): "))

    if choice == 1:
        w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
    elif choice == 2:
        infura_key = input("Infura APIキーを入力してください: ")
        w3 = Web3(Web3.HTTPProvider(f'https://sepolia.infura.io/v3/{infura_key}'))
    elif choice == 3:
        infura_key = input("Infura APIキーを入力してください: ")
        w3 = Web3(Web3.HTTPProvider(f'https://goerli.infura.io/v3/{infura_key}'))
    elif choice == 4:
        infura_key = input("Infura APIキーを入力してください: ")
        w3 = Web3(Web3.HTTPProvider(f'https://mainnet.infura.io/v3/{infura_key}'))
    elif choice == 5:
        rpc_url = input("RPC URLを入力してください: ")
        w3 = Web3(Web3.HTTPProvider(rpc_url))
    else:
        print("無効な選択です。デフォルトでGanacheに接続します。")
        w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))

    if not w3.is_connected():
        print("Ethereumネットワークに接続できませんでした。")
        return

    print(f"ネットワークに接続しました。Chain ID: {w3.eth.chain_id}")

    # 秘密鍵の設定
    private_key = input("秘密鍵を入力してください (0xプレフィックスなし): ")
    if private_key.startswith('0x'):
        private_key = private_key[2:]

    # アカウント設定
    account = w3.eth.account.from_key('0x' + private_key)
    w3.eth.default_account = account.address

    print(f"アカウントアドレス: {account.address}")
    print(f"残高: {w3.from_wei(w3.eth.get_balance(account.address), 'ether')} ETH")

    # トークン詳細の設定
    token_name = input("トークン名を入力してください: ")
    token_symbol = input("トークンシンボルを入力してください: ")
    token_decimals = int(input("小数点以下の桁数を入力してください (通常は18): "))
    initial_supply = int(input("初期供給量を入力してください: "))

    # コントラクトのコンパイルとデプロイ
    print("コントラクトをコンパイルしています...")
    contract_interface = compile_contract()

    # コントラクトオブジェクトの作成
    Contract = w3.eth.contract(
        abi=contract_interface['abi'],
        bytecode=contract_interface['bin']
    )

    # トランザクションの構築
    print("トランザクションを構築しています...")
    nonce = w3.eth.get_transaction_count(account.address)

    # ガス料金の見積もり
    gas_price = w3.eth.gas_price

    # コントラクトデプロイトランザクションの構築
    construct_txn = Contract.constructor(token_name, token_symbol, token_decimals, initial_supply).build_transaction({
        'from': account.address,
        'nonce': nonce,
        'gas': 2000000,
        'gasPrice': gas_price
    })

    # トランザクションに署名
    signed_txn = w3.eth.account.sign_transaction(construct_txn, '0x' + private_key)

    # トランザクションを送信
    print("トランザクションを送信しています...")
    tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)

    # トランザクション確認を待機
    print(f"トランザクションが送信されました。トランザクションハッシュ: {tx_hash.hex()}")
    print("トランザクションの確認を待っています...")
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    if tx_receipt.status == 1:
        print(f"トークンコントラクトが正常にデプロイされました！")
        print(f"コントラクトアドレス: {tx_receipt.contractAddress}")

        # コントラクト情報を表示
        token_contract = w3.eth.contract(
            address=tx_receipt.contractAddress,
            abi=contract_interface['abi']
        )

        token_info = {
            "コントラクトアドレス": tx_receipt.contractAddress,
            "トークン名": token_contract.functions.name().call(),
            "シンボル": token_contract.functions.symbol().call(),
            "小数点以下の桁数": token_contract.functions.decimals().call(),
            "総供給量": token_contract.functions.totalSupply().call() / (10 ** token_contract.functions.decimals().call()),
            "オーナーアドレス": token_contract.functions.owner().call()
        }

        print("\nトークン情報:")
        for key, value in token_info.items():
            print(f"{key}: {value}")

        # JSONファイルにABIと契約情報を保存
        output_data = {
            "network": w3.eth.chain_id,
            "contract_address": tx_receipt.contractAddress,
            "abi": contract_interface['abi'],
            "token_info": token_info
        }

        with open(f"{token_symbol}_token_info.json", "w") as f:
            json.dump(output_data, f, indent=4)

        print(f"\n{token_symbol}_token_info.jsonファイルにコントラクト情報が保存されました。")
        files.download(f"{token_symbol}_token_info.json")

        return token_contract, tx_receipt.contractAddress
    else:
        print("トランザクションが失敗しました。")
        return None, None

# トークン操作関数の定義
def token_operations(token_contract, token_address, w3, private_key):
    while True:
        print("\nトークン操作:")
        print("1. 残高を確認")
        print("2. トークンを送信")
        print("3. トークンを追加発行 (mint)")
        print("4. トークンを焼却 (burn)")
        print("5. 終了")

        op_choice = int(input("操作を選択してください (1-5): "))

        if op_choice == 1:
            address = input("残高を確認するアドレスを入力してください: ")
            balance = token_contract.functions.balanceOf(address).call()
            decimals = token_contract.functions.decimals().call()
            print(f"残高: {balance / (10 ** decimals)} {token_contract.functions.symbol().call()}")

        elif op_choice == 2:
            recipient = input("受取人のアドレスを入力してください: ")
            amount_str = input("送信する金額を入力してください: ")
            amount = int(float(amount_str) * (10 ** token_contract.functions.decimals().call()))

            nonce = w3.eth.get_transaction_count(w3.eth.default_account)
            txn = token_contract.functions.transfer(recipient, amount).build_transaction({
                'from': w3.eth.default_account,
                'nonce': nonce,
                'gas': 200000,
                'gasPrice': w3.eth.gas_price
            })

            signed_txn = w3.eth.account.sign_transaction(txn, private_key)
            tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
            print(f"トランザクションが送信されました。ハッシュ: {tx_hash.hex()}")

            tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
            if tx_receipt.status == 1:
                print("トークン転送が成功しました!")
            else:
                print("トークン転送が失敗しました。")

        elif op_choice == 3:
            if token_contract.functions.owner().call() != w3.eth.default_account:
                print("オーナーではないためミントできません。")
                continue

            to_address = input("トークンを発行する先のアドレスを入力してください: ")
            amount_str = input("発行する金額を入力してください: ")
            amount = int(amount_str)  # この関数ではトークン単位で入力

            nonce = w3.eth.get_transaction_count(w3.eth.default_account)
            txn = token_contract.functions.mint(to_address, amount).build_transaction({
                'from': w3.eth.default_account,
                'nonce': nonce,
                'gas': 200000,
                'gasPrice': w3.eth.gas_price
            })

            signed_txn = w3.eth.account.sign_transaction(txn, private_key)
            tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
            print(f"トランザクションが送信されました。ハッシュ: {tx_hash.hex()}")

            tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
            if tx_receipt.status == 1:
                print("トークンミントが成功しました!")
            else:
                print("トークンミントが失敗しました。")

        elif op_choice == 4:
            if token_contract.functions.owner().call() != w3.eth.default_account:
                print("オーナーではないためバーンできません。")
                continue

            amount_str = input("焼却する金額を入力してください: ")
            amount = int(amount_str)  # この関数ではトークン単位で入力

            nonce = w3.eth.get_transaction_count(w3.eth.default_account)
            txn = token_contract.functions.burn(amount).build_transaction({
                'from': w3.eth.default_account,
                'nonce': nonce,
                'gas': 200000,
                'gasPrice': w3.eth.gas_price
            })

            signed_txn = w3.eth.account.sign_transaction(txn, private_key)
            tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
            print(f"トランザクションが送信されました。ハッシュ: {tx_hash.hex()}")

            tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
            if tx_receipt.status == 1:
                print("トークンバーンが成功しました!")
            else:
                print("トークンバーンが失敗しました。")

        elif op_choice == 5:
            break

        else:
            print("無効な選択です。もう一度お試しください。")

# メイン実行関数
def main():
    print("🚀 Google Colaboratory用ERC20トークン発行プログラム 🚀")
    token_contract, token_address = deploy_token()

    if token_contract and token_address:
        choice = input("\nトークン操作を行いますか？ (y/n): ")
        if choice.lower() == 'y':
            private_key = input("秘密鍵を再入力してください (0xプレフィックスなし): ")
            if private_key.startswith('0x'):
                private_key = private_key[2:]
            w3 = token_contract.web3
            token_operations(token_contract, token_address, w3, '0x' + private_key)

    print("プログラムを終了します。")

# エントリーポイント
if __name__ == "__main__":
    main()